In [1]:
import os
import glob
import pandas as pd
import numpy as np

from transformers4rec import tf as tr
import tensorflow as tf
from transformers4rec.tf.ranking_metric import NDCGAt, RecallAt

In [2]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", '../data/')

In [3]:
from merlin_standard_lib import Schema
# define schema object to pass it to the TabularSeqeunceFeatures class
SCHEMA_PATH = os.path.join(INPUT_DATA_DIR, 'schema_test.pb')
schema = Schema().from_proto_text(SCHEMA_PATH)

In [4]:
sequence_length = 20
inputs = tr.TabularSequenceFeatures.from_schema(
schema,
max_sequence_length = sequence_length,
aggregation = 'concat',
)

In [5]:
inputs

TabularSequenceFeatures(
  (_aggregation): ConcatFeatures()
  (to_merge): Dict(
    (continuous_layer): ContinuousFeatures(timestamp/age_days/LogOp/Normalize/list, timestamp/day/list, timestamp/hour/cos/list, timestamp/hour/list, timestamp/hour/sin/list, timestamp/month/list, timestamp/weekday/cos/list, timestamp/weekday/list, timestamp/weekday/sin/list, timestamp/year/list, user_age)
    (categorical_layer): SequenceEmbeddingFeatures(
      (feature_config): Dict(
        (item_id/list): TableConfig(vocabulary_size=51997, dim=64, initializer=None, optimizer=None, combiner='mean', name='item_id/list')
        (category/list): TableConfig(vocabulary_size=333, dim=64, initializer=None, optimizer=None, combiner='mean', name='category/list')
        (user_country): TableConfig(vocabulary_size=63, dim=64, initializer=None, optimizer=None, combiner='mean', name='user_country')
      )
      (_pre): SequentialTabularTransformations(
        (layers): List(
          (0): FilterFeatures(
     

In [6]:
def tf_yoochoose_like():
    return tr.data.tabular_sequence_testing_data.tf_synthetic_data(
    num_rows=100, min_session_length=5, max_session_length=20
    )

In [7]:
data = tf_yoochoose_like()

2021-12-06 15:02:40.595818: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
data

{'session_id': <tf.Tensor: shape=(100,), dtype=int32, numpy=
 array([ 2636874,  8981008,   550188,  7898268,  6830425,  2988193,
         4801790,  9854728,  9072580,  6732889,  6121464,  8913957,
          416666,  5335197,  8126749,  9556836,  7832841,  7848731,
         8077730,  8825718,  6762587,  3743698,  5611785,  9496022,
         7629948,  4501697,  9135159,  4778357,  7987938,  6408256,
         4265154,  6941044,  8035739,  2769107,  6660315,  8465474,
         5616160,  9988821,  2944139, 11525584,  5920296,  8495015,
         6788167,  8171395,  4068166,  2162610,  6244120, 11302074,
          979640,  2338123,  6635974, 10012410, 11109169,  3239239,
         5560499,  3913750,  7711535,  3968813,  1243055,  2720170,
         3521881,  1824767, 11267492,  1831669,  4145433, 10329290,
        10491403,  1335307,  8487790,  2065081,  8029561,  2964219,
         4154340,  1692219, 10291226,  9798592, 10276274,  4003789,
         1598255,    52954,  5773099,  9507703,  952006

In [9]:
data = tf_yoochoose_like()
body = tr.SequentialBlock([inputs, tr.MLPBlock([64])])
targets = {"target": tf.cast(tf.random.uniform((100,), maxval=2, dtype=tf.int32), tf.float32)}
model = tr.BinaryClassificationTask("target").to_model(body, inputs)
model.compile(optimizer="adam", run_eagerly=False)
dataset = tf.data.Dataset.from_tensor_slices((data, targets)).batch(50)

In [19]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [11]:
data_as_list = list(dataset)

In [23]:
data_as_list[0][0]['session_id']

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([11366838,  1192789,   696939,  3390264,  2392789, 11558118,
        2023645,  8257447,   395006,  2192115,  9898031,  8625090,
       10301031,  5040198,  9000713, 11301764,  8905242,  2195505,
        6415859,  9188125,   958044,  4003791,  9611781,  4459846,
        8085295,   237393,  5956030,  2939286, 10635378,  4284968,
        7299163,  7460699,  1732437,  7411932,  7431694,  7842074,
        2293448,  2678465,  9659383, 11215737,  8415960,   518407,
        2425579,  2166102,  8733643,  3533019,  3657844, 11002482,
        4842298, 10986411], dtype=int32)>

In [11]:
losses = model.fit(dataset, epochs=5)

Epoch 1/5
2/2 [==============================] - 3s 21ms/step - target/binary_classification_task/precision: 0.4433 - target/binary_classification_task/recall: 0.9348 - target/binary_classification_task/binary_accuracy: 0.4300 - target/binary_classification_task/auc: 0.4628 - loss: 0.7056 - regularization_loss: 0.0000e+00 - total_loss: 0.7056
Epoch 2/5
2/2 [==============================] - 0s 19ms/step - target/binary_classification_task/precision: 0.5000 - target/binary_classification_task/recall: 0.8696 - target/binary_classification_task/binary_accuracy: 0.5400 - target/binary_classification_task/auc: 0.6184 - loss: 0.6899 - regularization_loss: 0.0000e+00 - total_loss: 0.6899
Epoch 3/5
2/2 [==============================] - 0s 25ms/step - target/binary_classification_task/precision: 0.6512 - target/binary_classification_task/recall: 0.6087 - target/binary_classification_task/binary_accuracy: 0.6700 - target/binary_classification_task/auc: 0.7081 - loss: 0.6773 - regularization_los

In [10]:
data = list(dataset)

In [23]:
type(data[0])

tuple

In [31]:
type(data[1][0]['session_start'])

tensorflow.python.framework.ops.EagerTensor

In [20]:
dataset

<BatchDataset shapes: ({session_id: (None,), session_start: (None,), timestamp/age_days/list: (None, 20), timestamp/age_days/LogOp/Normalize/list: (None, 20), timestamp/hour/list: (None, 20), timestamp/weekday/list: (None, 20), timestamp/day/list: (None, 20), timestamp/month/list: (None, 20), timestamp/year/list: (None, 20), timestamp/hour/sin/list: (None, 20), timestamp/hour/cos/list: (None, 20), timestamp/weekday/sin/list: (None, 20), timestamp/weekday/cos/list: (None, 20), ts/list: (None, 20), ts/last: (None,), ts/first: (None,), item_id/list: (None, 20), session_size: (None,), category/list: (None, 20), day_idx: (None,), user_country: (None,), user_age: (None,)}, {target: (None,)}), types: ({session_id: tf.int32, session_start: tf.float32, timestamp/age_days/list: tf.int32, timestamp/age_days/LogOp/Normalize/list: tf.float32, timestamp/hour/list: tf.float32, timestamp/weekday/list: tf.float32, timestamp/day/list: tf.float32, timestamp/month/list: tf.float32, timestamp/year/list: tf

In [12]:
data

{'session_id': <tf.Tensor: shape=(100,), dtype=int32, numpy=
 array([  711535,  7634525,  1064431,   750313, 10667252,  5432563,
         3385050,  9540494, 10679201,  9763539,   936101,  1569883,
         7002310,  2392808, 10419228,  6498305,  2142928,  5410061,
          493497,   698941,  3619510,   745708,  9856810, 11050686,
          512184,  6088159,  9345410,  2987543,  3499587,  9127776,
        11045228,  3430627,  3064350,  9392894,  1763212,   425805,
         8940034,  6700220,  3784166,  9862134,  7561840,  7551623,
         5004913, 11504839,  3978375,  4906280,  5441084,  1109172,
         7096091,  4149691,  4337089,  1520295,  9050394,  9506468,
         8625085,  4098960, 10224741,  5856721,  3046863, 10679033,
        10952984,  2117142,  6295730,   800148, 10784329,  9217077,
         6337262,  4833798,  1775300,  2799002,  6540691,  7892403,
         4493530,  7407470, 10382337,  4788931,  9935895, 10603751,
         4093801,  6114690,  7881153,  2306267,  416719